In [15]:
import nltk
import unicodedata
import numpy as np
import pandas as pd
from joblib import dump
from nltk.stem import ISRIStemmer, WordNetLemmatizer
from gensim.models import FastText
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models, regularizers


In [2]:
file_path = "train.xlsx" 
df_train = pd.read_excel(file_path)
df_train

,review_description,rating
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1.0
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1.0
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1.0
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1.0
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1.0
...,...,...
32031,التطبيق اصبح سيء للغايه نقوم بطلب لا يتم وصول ...,-1.0
32032,y love you,1.0
32033,الباقه بتخلص وبشحن مرتين باقه اضافيه ١٠٠ جنيه,-1.0
32034,تطبيق فاشل وصلني الطلب ناقص ومش ينفع اعمل حاجة...,-1.0


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32036 entries, 0 to 32035
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   review_description  32036 non-null  object 
 1   rating              32036 non-null  float64
dtypes: float64(1), object(1)
memory usage: 500.7+ KB


In [4]:
df_train.isnull().sum()

review_description    0
rating                0
dtype: int64

In [5]:
arabic_stop_words = [
    "و", "في", "من", "على", "إلى", "لا", "أو", "هو", "هي", "يكون",
    "أنا", "أنت", "هو", "هي", "نحن", "أنتم", "هم",
    "عن", "مع", "كما", "مثل", "بين", "إذا", "حتى", "منذ",
    "و", "أو", "لكن", "إذا", "إن",
    "اليوم", "غداً", "الآن", "ثم", "بعد",
    "كان", "يكون", "أصبح", "صار", "ليس", "لم",
    "هذا", "هذه", "ذلك", "تلك", 
    "كل", "على", "فيه", "منه", "عنه", "له", "به", "إليه", "لها", "فيها",
    "بها", "منها", "عنها", "إليها", "الذي", "التي", "اللذين", "اللذان", "اللتان",
    "اللتين", "هؤلاء", "ذلك", "هذه", "هذا", "تلك", "تحت", "فوق", "معه", "لديه",
    "عليه", "عليها", "أي", "هل", "إذا", "ماذا", "هناك", "هنالك", "إلى",
    "يناير", "فبراير", "مارس", "إبريل", "مايو", "يونيو", "يوليو", "أغسطس", "سبتمبر", "أكتوبر", "نوفمبر", "ديسمبر",
    "الأحد", "الاثنين", "الثلاثاء", "الأربعاء", "الخميس", "الجمعة", "السبت"
]

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import string


def text_cleaning(text, stemmer=ISRIStemmer(), lemmatizer=WordNetLemmatizer()):
    # remove stop words and punctuation
    text = text.lower()
    tokenizer = RegexpTokenizer(r"\w+")
    words = tokenizer.tokenize(text)

    stop_words = set(stopwords.words("english")).union(
        set(string.punctuation).union(set(arabic_stop_words))
    )

    filtered_list = [word for word in words if word.casefold() not in stop_words]

    # word stemming
    stem_words = [stemmer.stem(word) for word in filtered_list]

    # lemmatized_words = [lemmatizer.lemmatize(word)for word in filtered_list]

    return " ".join(map(str, stem_words))  # + lemmatized_words


df_train["new review_description"] = df_train["review_description"].apply(
    lambda text: text_cleaning(text)
)
df_train.head(20)

,review_description,rating,new review_description
0,شركه زباله و سواقين بتبرشم و مفيش حتي رقم للشك...,-1.0,شرك زبل سوق رشم فيش حتي رقم شكو سوق يسب يمش مي...
1,خدمة الدفع عن طريق الكي نت توقفت عندي اصبح فقط...,1.0,خدم دفع طرق الك نت وقف عند اصبح فقط دفع نقد
2,تطبيق غبي و جاري حذفه ، عاملين اكواد خصم و لما...,-1.0,طبق غبي جري حذف عمل كود خصم لما خدم كتر مرة عم...
3,فعلا تطبيق ممتاز بس لو فى امكانية يتيح لمستخدم...,1.0,فعل طبق متز بس لو فى مكن يتح خدم طبق ان تطع غي...
4,سيء جدا ، اسعار رسوم التوصيل لا تمت للواقع ب ص...,-1.0,سيء جدا سعر رسم وصل تمت وقع ب صله
5,قعد عشرين سنة يدور على سائق بس اما عن توصيل ال...,0.0,قعد عشر سنة يدر سئق بس اما وصل شيء جيد جدا
6,احلئ تطبيق,1.0,حلئ طبق
7,رائع واو مدهش,1.0,رئع واو دهش
8,مکو بس البحرین وعمان وغیرهه بس العراق مکو یعنی...,-1.0,مکو بس حری وعم غیر بس عرق مکو یعنی نجم وحد علی...
9,تطبيق جميل يستاهل الخمس نجوم👍👍👍,1.0,طبق جمل تهل خمس نجم


In [7]:
tfidf = TfidfVectorizer(analyzer="word",max_features=10000, use_idf=True)
tfidf.fit(df_train["new review_description"])
X = tfidf.transform(df_train["new review_description"])

terms = tfidf.get_feature_names_out()
X1_print = pd.DataFrame(X.toarray(), columns=terms)
print(X1_print.shape)
X1_print.head()

(32036, 10000)


,00,000,00000⁰00مو,000omr,007,00l8l90l,011,0155_,064ssr100xtjuenezqg2ftbkoa6vg91pi21273620,08,...,ﻹعل,ﻻحظ,ﻻزم,ﻻسباب,ﻻسف,ﻻصل,ﻻطﻻق,ﻻعل,ﻻنه,ﻻيتم
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
X = X.toarray()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
y = np.array(df_train["rating"])
y

array([-1.,  1., -1., ..., -1., -1.,  1.])

In [10]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=1000, random_state=42, stratify=y
)
print("X_train shape :", X_train.shape)
print("y_train shape :", y_train.shape)
print("X_val shape :", X_val.shape)
print("y_val shape :", y_val.shape)

X_train shape : (31036, 10000)
y_train shape : (31036,)
X_val shape : (1000, 10000)
y_val shape : (1000,)


In [11]:
num_classes = 3

X_train_reshaped = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_val_reshaped = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))

y_train_shifted = y_train + 1
y_val_shifted = y_val + 1

In [16]:
model = Sequential()

# Adjust the input_shape based on the features in your data
model.add(SimpleRNN(64, activation="tanh", input_shape=(1, X_train.shape[1])))
model.add(tf.keras.layers.Dropout(0.6))
model.add(Dense(32, activation="tanh", kernel_regularizer=regularizers.l2(0.01),
    bias_regularizer=regularizers.l2(0.01)))
model.add(Dense(16, activation="tanh"))
# Output layer with softmax activation for multi-class classification
model.add(Dense(num_classes, activation="softmax"))

# Compile the model with 'sparse_categorical_crossentropy' loss
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

checkpoint1 = ModelCheckpoint(
    filepath="RNN.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

In [17]:
# Train the model
model.fit(
    X_train_reshaped,
    y_train_shifted,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_reshaped, y_val_shifted),
    callbacks=[checkpoint1],
)

Epoch 1/10
970/970 [==============================] - 12s 11ms/step - loss: 0.6053 - accuracy: 0.7937 - val_loss: 0.4610 - val_accuracy: 0.8430
Epoch 2/10
970/970 [==============================] - 10s 10ms/step - loss: 0.4500 - accuracy: 0.8466 - val_loss: 0.4580 - val_accuracy: 0.8310
Epoch 3/10
970/970 [==============================] - 9s 10ms/step - loss: 0.4125 - accuracy: 0.8598 - val_loss: 0.4648 - val_accuracy: 0.8290
Epoch 4/10
970/970 [==============================] - 9s 10ms/step - loss: 0.3855 - accuracy: 0.8689 - val_loss: 0.4699 - val_accuracy: 0.8260
Epoch 5/10
970/970 [==============================] - 11s 11ms/step - loss: 0.3615 - accuracy: 0.8779 - val_loss: 0.4972 - val_accuracy: 0.8180
Epoch 6/10
970/970 [==============================] - 10s 10ms/step - loss: 0.3457 - accuracy: 0.8837 - val_loss: 0.5031 - val_accuracy: 0.8130
Epoch 7/10
970/970 [==============================] - 10s 10ms/step - loss: 0.3261 - accuracy: 0.8902 - val_loss: 0.5165 - val_accuracy: 0

In [18]:
model.load_weights("RNN.hdf5")

# Evaluate the model
loss, accuracy = model.evaluate(X_val_reshaped, y_val_shifted)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

32/32 [==============================] - 0s 3ms/step - loss: 0.4610 - accuracy: 0.8430
Validation Loss: 0.4609745442867279, Validation Accuracy: 0.8429999947547913


In [25]:
model2 = Sequential()

model2.add(LSTM(64, activation="tanh", dropout=0.5))
model2.add(Dense(32, activation="tanh"))
model2.add(Dense(num_classes, activation="softmax"))

model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

checkpoint2 = ModelCheckpoint(
    filepath="LSTM.hdf5",
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

In [26]:
# Train the model
model2.fit(
    X_train_reshaped,
    y_train_shifted,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_reshaped, y_val_shifted),
    callbacks=[checkpoint2],
)

Epoch 1/10
970/970 [==============================] - 36s 34ms/step - loss: 0.5958 - accuracy: 0.7616 - val_loss: 0.4726 - val_accuracy: 0.8320
Epoch 2/10
970/970 [==============================] - 34s 35ms/step - loss: 0.5179 - accuracy: 0.7996 - val_loss: 0.4700 - val_accuracy: 0.8270
Epoch 3/10
970/970 [==============================] - 33s 34ms/step - loss: 0.4870 - accuracy: 0.8101 - val_loss: 0.4746 - val_accuracy: 0.8280
Epoch 4/10
970/970 [==============================] - 32s 33ms/step - loss: 0.4577 - accuracy: 0.8222 - val_loss: 0.4813 - val_accuracy: 0.8200
Epoch 5/10
970/970 [==============================] - 32s 33ms/step - loss: 0.4372 - accuracy: 0.8318 - val_loss: 0.4893 - val_accuracy: 0.8190
Epoch 6/10
970/970 [==============================] - 35s 36ms/step - loss: 0.4139 - accuracy: 0.8430 - val_loss: 0.4999 - val_accuracy: 0.8160
Epoch 7/10
970/970 [==============================] - 32s 33ms/step - loss: 0.3976 - accuracy: 0.8461 - val_loss: 0.5052 - val_accuracy:

In [27]:
model2.load_weights("LSTM.hdf5")

# Evaluate the model
loss, accuracy = model2.evaluate(X_val_reshaped, y_val_shifted)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

32/32 [==============================] - 0s 5ms/step - loss: 0.4726 - accuracy: 0.8320
Validation Loss: 0.4726279079914093, Validation Accuracy: 0.8320000171661377


In [28]:
file_path = "test _no_label.csv"
df_test = pd.read_csv(file_path)
df_test

,ID,review_description
0,1,اهنئكم على خدمه العملاء في المحادثه المباشره م...
1,2,ممتاز جدا ولكن اتمنى ان تكون هناك بعض المسابقا...
2,3,كل محملته يقول تم ايقاف حطيت2 عشان تسوون الخطاء
3,4,شغل طيب
4,5,بعد ماجربت
...,...,...
995,996,يستهل
996,997,خدمة سيئة بكل المعايير
997,998,لؤي٠٣٣٢لؤ٣٤٣س
998,999,تطبيق غير صادق ف خصم الكوبونات


In [29]:
df_test["review_description"] = df_test["review_description"].apply(
    lambda text: text_cleaning(text)
)
df_test

,ID,review_description
0,1,هنئ خدم عملاء حدث بشر ما قصرو الل وفق يعط الف عفي
1,2,متز جدا ولكن منى ان تكون بعض سبق جئز طلب سعد ف...
2,3,حمل يقل تم يقف حطيت2 عشن تسو خطء
3,4,شغل طيب
4,5,اجرب
...,...,...
995,996,سهل
996,997,خدم سيئ بكل معايير
997,998,لؤي٠٣٣٢لؤ٣٤٣س
998,999,طبق غير صدق ف خصم كوبو


In [30]:
X_test = tfidf.transform(df_test["review_description"])
X_test = X_test.toarray()
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [31]:
y_pred_lstm = model2.predict(X_test_reshaped)
y_pred_rnn = model.predict(X_test_reshaped)

32/32 [==============================] - 0s 3ms/step


In [32]:
def return_prediction(y_pred):
    prediction = []
    for predict in y_pred_lstm:
        maxi = np.argmax(predict)
        if maxi == 2:
            prediction.append(1)
        elif maxi == 1:
            prediction.append(0)
        else:
            prediction.append(-1)
    return prediction


prediction_lstm = return_prediction(y_pred_lstm)
prediction_rnn = return_prediction(y_pred_rnn)

print(len(prediction_lstm), len(prediction_rnn))

1000 1000


In [33]:
results_df = pd.DataFrame({'ID': df_test['ID'], 'rating': prediction_lstm})
results_df.to_csv('test_results_LSTM.csv', index=False)
results_df

,ID,rating
0,1,1
1,2,1
2,3,-1
3,4,1
4,5,1
...,...,...
995,996,1
996,997,-1
997,998,1
998,999,-1


In [34]:
results_df = pd.DataFrame({'ID': df_test['ID'], 'rating': prediction_rnn})
results_df.to_csv('test_results_rnn.csv', index=False)
results_df

,ID,rating
0,1,1
1,2,1
2,3,-1
3,4,1
4,5,1
...,...,...
995,996,1
996,997,-1
997,998,1
998,999,-1


In [35]:
comparison_result = np.array(prediction_lstm) == np.array(prediction_rnn)
print("The equal values are : ", np.sum(comparison_result))

The equal values are :  1000
